# DemoDay Presentation - New DAGNode Structure

### Rationale

    Discussion in https://github.com/Qiskit/qiskit-terra/issues/6493 
      "Performance regression caused by DAGNode op property"
    
    Performance issues when using @property for 'op'
    
    Desire to remove 'type' attribute and checks of 'node.type == '
    
    Avoid using '_op' and other ‘private’ variables in internal code

### Previous State

    DAGNode attributes
        type - ['in', 'out', 'op']
        op - Instruction (node.type == 'op' only)
        qargs
        cargs
        wire - Qubit or Clbit (node.type in ['in', 'out'] only)
        node_id
        sort_key - used in DAGCircuit.topological_nodes()
        name - as @property
        condition - as @property

### DAGCircuit / DAGNode function

    DAGCircuit instance created directly or through circuit_to_dag

    DAGNode ‘op’ nodes not created directly by user
    
        'apply_operation_front' and 'apply_operation_back' take ‘op’ arg
        DAGNodes created in '_add_op_node' calls
        
	DAGNode ‘in’ and ‘out’ nodes not created directly by user
    
        'add_qubits' and 'add_clbits' take ‘qubits’ and ‘clbits’ args
        DAGNodes created in '_add_wire' calls
        
	With 2 exceptions (sabre_swap and lookahead_swap) DAGNodes only created
        in '_add_op_node' and '_add_wire'
    
	Users should only be calling 'apply_operation_back', etc. and not
        instantiating DAGNodes directly

### Proposed Solution

	Change DAGNode from primary class to parent class with attributes
        node_id
        condition (already deprecated)
        name (to be deprecated)
        
	Add DAGOpNode class with attributes
        op
        qargs
        cargs
        sort_key
        
	Add DAGInNode and DAGOutNode classes with attributes
        wire
        sort_key

### New Commit #6567

    Implement new DAGNode classes

    Internally, all ‘if node.type == op’ replaced by ‘if isinstance(node, DAGOpNode)’
        and similarly for DAGInNode and DAGOutNode
        
    Deprecations in DAGNode
    
    Many doc-string changes
    
    DAGCircuit, circuit_to_dag, and dag_to_circuit function exactly as before

### Deprecations

    These DAGNode kwargs are being deprecated as of 0.19.0
        type
        op
        name
        qargs
        cargs
        wire
        
	These DAGNode attributes are being deprecated as of 0.19.0
        type
        op
        qargs
        cargs
        wire
        
	The DAGNode ‘condition’ kwarg has been deprecated and removed,
        and the ‘condition’ attribute was deprecated in 0.18.0

### Performance

	Performance benchmarks run after the PR was merged showed mostly small improvements.
    
	All the transpiler tests were improved by a few percent
    
	Small benchmark decresase in performance in circuit_to_dag. May be the overhead
        for the deprecation checks in DAGNode. Needs further investigating.

### Future To Do

	Remove ‘condition’ attribute 3 months after 0.18.0 release date
    
	Remove all other deprecated items 3 months after release of 0.19.0
    
	Deprecate ‘name’ attribute and convert all ‘node.name’ references to ‘node.op.name’

In [ ]:
from qiskit.circuit import QuantumCircuit
from qiskit.visualization import dag_drawer
from qiskit.converters import circuit_to_dag

# Create basic Bell circuit

qc = QuantumCircuit(2, 1)
qc.x(1)
qc.h(0)
qc.cx(0, 1)

display(qc.draw('mpl', cregbundle=False))
print(qc._data)

In [ ]:
# Convert circuit to dag and display dag

dag = circuit_to_dag(qc)
display(dag_drawer(dag))

In [ ]:
# Using pre-6567 branch

for node in dag.topological_nodes():
    if node.type == "op":
        print(type(node), node.type, node.op)
    else:
        print(type(node), node.type, node.wire)

In [ ]:
# Using post-6567 branch

from qiskit.dagcircuit import DAGOpNode

for node in dag.topological_nodes():
    if isinstance(node, DAGOpNode):
        print(type(node), node.op)
    else:
        print(type(node), node.wire)

In [ ]:
<class 'qiskit.dagcircuit.dagnode.DAGInNode'> Qubit(QuantumRegister(2, 'q'), 0)
<class 'qiskit.dagcircuit.dagnode.DAGOpNode'> <qiskit.circuit.library.standard_gates.h.HGate object at 0x7f65b6f89d30>
<class 'qiskit.dagcircuit.dagnode.DAGInNode'> Qubit(QuantumRegister(2, 'q'), 1)
<class 'qiskit.dagcircuit.dagnode.DAGOpNode'> <qiskit.circuit.library.standard_gates.x.XGate object at 0x7f65b6f89fd0>
<class 'qiskit.dagcircuit.dagnode.DAGOpNode'> <qiskit.circuit.library.standard_gates.x.CXGate object at 0x7f65b6f89400>
<class 'qiskit.dagcircuit.dagnode.DAGOutNode'> Qubit(QuantumRegister(2, 'q'), 0)
<class 'qiskit.dagcircuit.dagnode.DAGOutNode'> Qubit(QuantumRegister(2, 'q'), 1)
<class 'qiskit.dagcircuit.dagnode.DAGInNode'> Clbit(ClassicalRegister(1, 'c'), 0)
<class 'qiskit.dagcircuit.dagnode.DAGOutNode'> Clbit(ClassicalRegister(1, 'c'), 0)